In [1]:
#Import necessory libraries
import selenium
import pandas as pd
import time
import  requests
from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Connecting to the webdriver
driver = webdriver.Chrome(r"C:\Users\abhic\Downloads\chromedriver_win32 (2)\chromedriver.exe")

In [4]:
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)
#clicking on cancel buttopn 
driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input").click()

In [5]:
srch_items = ['laptops', 'Phones','smart watches']

In [6]:
title = []
review_text = []
ratings = []

In [7]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [8]:
urls=[]

In [9]:
for i in srch_items:
    #Find search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    #clicking on search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [10]:
len(urls)

288

In [11]:
for i in urls:
    driver.get(i)
    time.sleep(3)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,4000)")
        time.sleep(1)
        #clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(2)
            scrap()
    except: continue

In [12]:
len(ratings), len(review_text), len(title)

(23405, 23405, 23405)

In [13]:
#creating a dataframe
data = list(zip(title,review_text,ratings))
df2 = pd.DataFrame(data, columns = ["Review_title","Reiew_text","Ratings"])
df2


,Review_title,Reiew_text,Ratings
0,Really Nice,"Performance wise too good, display quality is ...",4
1,Mind-blowing purchase,Awesome laptop 😍\nBoot very fast and every app...,5
2,Terrific,Compact design and great performance,5
3,Best in the market!,The product is very good.\nJust loved it at th...,5
4,Good choice,1.Product was good without any damages.\n2.Lap...,4
...,...,...,...
23400,Brilliant,Nice love you❤😘🤟 beautiful and very very much😊...,5
23401,Highly recommended,It's amazing products,5
23402,Wonderful,It's a very good products\nDelivery is fast,5
23403,Perfect product!,Good quality worth for money,5


In [14]:
df2.to_csv("Rating_re.csv")